In [1]:
from typing import List, Iterable

import numpy as np
import matplotlib.pyplot as plt

In [2]:
import abc

class BaseLoss(abc.ABC):
    @abc.abstractmethod
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        raise NotImplementedError

    @abc.abstractmethod
    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        raise NotImplementedError

In [3]:
class MSELoss(BaseLoss):
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        return np.mean(np.square(np.dot(X,w)-y))
        
    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        return 2*np.dot(X.T, np.dot(X,w)-y)/len(X)

In [4]:
def gradient_descent(w_init: np.ndarray, X: np.ndarray, y: np.ndarray, 
                        loss: BaseLoss, lr: float, n_iterations: int = 100000) -> List[np.ndarray]:
    w_gd = np.array([w_init])
    for n in range(n_iterations):
        w_gd = np.append(w_gd, np.array([w_gd[n]-lr*loss.calc_grad(X, y, w_gd[n])]), axis=0)
    return w_gd

In [12]:
np.random.seed(1337)

n_features = 2
n_objects = 300
batch_size = 10
num_steps = 43

w_true = np.random.normal(size=(n_features, ))

X = np.random.uniform(-5, 5, (n_objects, n_features))
X *= (np.arange(n_features) * 2 + 1)[np.newaxis, :]  
y = X.dot(w_true) + np.random.normal(0, 1, (n_objects))
w_init = np.random.uniform(-2, 2, (n_features))

print(X.shape)
print(y.shape)

(300, 2)
(300,)


In [15]:
loss = MSELoss()
w_list = gradient_descent(w_init, X, y, loss, 0.01, 100)
print(loss.calc_loss(X, y, w_list[0]))
print(loss.calc_loss(X, y, w_list[-1]))

425.58917680450253
0.8670644395649493


In [16]:
class LinearRegression:
    def __init__(self, loss: BaseLoss, lr: float = 0.01) -> None:
        self.loss = loss
        self.lr = lr
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> 'LinearRegression':
        X = np.asarray(X)
        y = np.asarray(y)
        X = np.hstack([X, np.ones([X.shape[0], 1])])
        w_reg = np.random.uniform(-2, 2, len(X[0]))
        self.w = gradient_descent(w_reg, X, y, self.loss, self.lr, 1000)
        return self
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        assert hasattr(self, "w"), "Linear regression must be fitted first"
        X = np.hstack([X, np.ones([X.shape[0], 1])])
        return np.dot(X,self.w[-1])

In [17]:
linear_regression = LinearRegression(MSELoss())

In [18]:
import pandas as pd

X_raw = pd.read_csv(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data", 
    header=None, 
    na_values=["?"]
)
X_raw.head()
X_raw = X_raw[~X_raw[25].isna()].reset_index()

In [19]:
y = X_raw[25]
X_raw = X_raw.drop(25, axis=1)

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [21]:
cat_features_mask = (X_raw.dtypes == "object").values
X_real = X_raw[X_raw.columns[~cat_features_mask]]
mis_replacer = SimpleImputer(strategy="mean")
X_no_mis_real = pd.DataFrame(data=mis_replacer.fit_transform(X_real), columns=X_real.columns)

X_cat = X_raw[X_raw.columns[cat_features_mask]].fillna("")
X_raw = pd.concat([X_no_mis_real, X_cat], axis=1)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [22]:
X_raw = pd.get_dummies(X_raw, drop_first=True)

In [23]:
X_train, X_test, y_train, y_test  = train_test_split(X_raw, y, \
                                                     test_size=0.3,\
                                                     shuffle=True,
                                                     random_state=0)

In [24]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [25]:
linear_regression.fit(X_train, y_train)

In [26]:
y_pred = linear_regression.predict(X_test)

In [27]:
from sklearn.metrics import mean_squared_error

In [28]:
mean_squared_error(y_test, y_pred)

17634105.35489321

In [29]:
class MSEL2Loss(BaseLoss):
    def __init__(self, coef: float = 1.):
        self.coef = coef
    
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        return np.mean(np.square(np.dot(X,w)-y))+self.coef*np.square(np.delete(w,-1,0))
        
    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        return 2*self.coef*w+2*np.dot(X.T, np.dot(X,w)-y)/len(X)

In [30]:
linear_regression = LinearRegression(MSEL2Loss(0.0001))

In [31]:
linear_regression.fit(X_train, y_train)

In [32]:
y_pred = linear_regression.predict(X_test)

In [33]:
mean_squared_error(y_test, y_pred)

17633049.041108135

In [34]:
import numpy.linalg as la

In [35]:
class HuberLoss(BaseLoss):
    def __init__(self, eps: float) -> None:
        self.eps = eps
    
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        grid = np.dot(X,w)-y
        return np.mean(
            np.where(np.abs(grid)<=self.eps,
                 0.5*np.square(grid),
                 self.eps*(np.abs(grid)-0.5*self.eps)))

    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        grid = np.dot(X,w)-y
        return np.mean(np.sum(np.dot(X.T,
            np.where(np.abs(grid)<=self.eps,
                 grid,
                 self.eps*np.sign(grid)))))
                 
        # if (la.norm(y - np.dot(X,w))) <= self.eps:
        #     return np.dot(X.T, (np.dot(X,self.w) - y))/y.shape[0]
        # else:
        #     return self.eps * np.dot(X.T, np.sign(np.dot(X,w) - y))/y.shape[0]

In [36]:
linear_regression = LinearRegression(HuberLoss(100.01))

In [37]:
linear_regression.fit(X_train, y_train)

In [38]:
y_pred = linear_regression.predict(X_test)

In [39]:
mean_squared_error(y_test, y_pred)

192633278.55214122